In [21]:
#%pip install tensorflow
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from utils.image_treatment import preprocess_mediapipe_landmarks

In [22]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [23]:
model_save_path = 'data/keypoint_classifier.keras'
model = tf.keras.models.load_model(model_save_path)
NUM_CLASSES = model.output_shape[-1]
NUM_CLASSES

8

In [25]:
GESTURE_LABELS = {
    0: 'Forward',
    1: 'Stop',
    2: 'Up',
    3: 'Land',
    4: 'Down',
    5: 'Back',
    6: 'Left',
    7: 'Right'
}

In [26]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip horizontal pour effet miroir
        frame = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Affichage des points
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Préparer vecteur d’entrée
                h, w = frame.shape[:2]
                input_vector = preprocess_mediapipe_landmarks(hand_landmarks, w, h)
                input_vector = np.expand_dims(input_vector, axis=0)  # (1, 42)
                print("input is", input_vector)
                # Prédiction
                prediction = model.predict(input_vector)
                pred_class = int(np.argmax(prediction))
                print("prediction is", pred_class)
                pred_label = GESTURE_LABELS.get(pred_class, 'Unknown')

                # Affichage sur la frame
                cv2.putText(frame, f'Gesture: {pred_label}', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                #print(f"Gesture reconnu : {pred_label}")

        cv2.imshow('Hand Gesture Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

input is [[ 0.          0.         -0.24861878 -0.08839779 -0.45856354 -0.320442
  -0.5801105  -0.49171272 -0.67403316 -0.64640886 -0.30939227 -0.68508285
  -0.42541435 -0.9392265  -0.55801105 -0.93370163 -0.62983423 -0.8453039
  -0.19889502 -0.7237569  -0.31491712 -1.         -0.45856354 -0.97790056
  -0.5414365  -0.878453   -0.0718232  -0.71270716 -0.16574585 -0.98895025
  -0.320442   -0.9723757  -0.42541435 -0.89502764  0.06629834 -0.6629834
  -0.01657459 -0.8729282  -0.13812155 -0.9116022  -0.23756906 -0.9005525 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
prediction is 1
input is [[ 0.          0.         -0.22834645 -0.12204725 -0.4055118  -0.26377952
  -0.53937006 -0.37007874 -0.6692913  -0.42519686 -0.27165353 -0.53543305
  -0.33070865 -0.70866144 -0.36220473 -0.8228347  -0.38188976 -0.9251968
  -0.15354331 -0.5748032  -0.17716536 -0.7677165  -0.18503937 -0.8897638
  -0.19291338 -0.992126   -0.03543307 -0.5826772  -0.03543307 -0.7598425
  -0.03543307 -0.88582677 -0.03937008 -1.   

KeyboardInterrupt: 